In [1]:
import pandas as pd
import os
import numpy as np
import glob
import gdal
from gdalconst import GA_ReadOnly


In [ ]:
idir = "R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_prj\\"
ndvi_list = glob.glob(idir+"*.tif")
ndvi_list.sort()

In [3]:
maskDs = gdal.Open('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_final\\eMODIS200001.tif', GA_ReadOnly)# your mask raster
projection=maskDs.GetProjectionRef()
geoTransform = maskDs.GetGeoTransform()
minx = geoTransform[0]
maxy = geoTransform[3]
maxx = minx + geoTransform[1] * maskDs.RasterXSize
miny = maxy + geoTransform[5] * maskDs.RasterYSize

In [ ]:
for f in ndvi_list:
    fname = os.path.basename(f)
    data=gdal.Open(f, GA_ReadOnly) #Your data the one you want to clip
    output = "R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_mask\\"+fname #output file
    gdal.Translate(output,data,format='GTiff',projWin=[minx,maxy,maxx,miny],outputSRS=projection)

In [ ]:
ndvi_list = glob.glob("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_mask\\"+"*.tif")
ndvi_list.sort()

In [ ]:
driver = gdal.GetDriverByName("GTiff")
for f in ndvi_list:
    time = os.path.basename(f)[-10:-4]
    ds = gdal.Open(f)
    arr = ds.ReadAsArray().astype(np.float)
    arr[arr==32767] = np.nan
    arr[arr==-2000] = np.nan
    arr = 0.0001*arr
    ndvi = driver.Create("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_final\\eMODIS" + time +'.tif', arr.shape[1], arr.shape[0], 1, gdal.GDT_Float32)
    ndvi.GetRasterBand(1).WriteArray(arr)
    ndvi.SetGeoTransform(geoTransform)
    ndvi.SetProjection(projection)
    ndvi.FlushCache()

In [5]:
#correct the missing value/ replace 202013
f = "E:\\New_ForDRI\\missing_NDVI\\NDVI_202012.tif"
output = "E:\\New_ForDRI\\missing_NDVI\\NDVI_202012_mask.tif" #output file
data=gdal.Open(f, GA_ReadOnly)
gdal.Translate(output,data,format='GTiff',projWin=[minx,maxy,maxx,miny],outputSRS=projection)


In [4]:
#correct the missing value/ replace 202013
driver = gdal.GetDriverByName("GTiff")
output = "E:\\New_ForDRI\\missing_NDVI\\NDVI_202012_mask.tif"
time = os.path.basename(output)[-10:-4]
ds = gdal.Open(output)
arr = ds.ReadAsArray().astype(np.float)
arr[arr==32767] = np.nan
arr[arr==-2000] = np.nan
arr = 0.0001*arr
ndvi = driver.Create("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_final\\eMODIS202012.tif", arr.shape[1], arr.shape[0], 1, gdal.GDT_Float32)
ndvi.GetRasterBand(1).WriteArray(arr)
ndvi.SetGeoTransform(geoTransform)
ndvi.SetProjection(projection)
ndvi.FlushCache()

In [4]:
ndvi_list =  glob.glob("R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_final\\"+"*.tif")
ndvi_list.sort()

In [5]:
ndvi_arr = []
for ndvi in ndvi_list:
    ds = gdal.Open(ndvi)
    arr = ds.ReadAsArray()
    arr[arr<-2] = np.nan
    ndvi_arr.append(arr)
ndvi_arr = np.asarray(ndvi_arr)

In [6]:
idir = "R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\"
driver = gdal.GetDriverByName("GTiff")
for i in range(1,54):
    if i < 10:
        week = '0'+str(i)
    else:
        week = str(i)
    ndvi_month = ndvi_arr[i-1::52,:,:]
    ndvi_mean = np.mean(ndvi_month,0)
    mean = driver.Create(idir+'NDVI_MEAN\\MEAN_' + week +'.tif', ndvi_mean.shape[1], ndvi_mean.shape[0], 1, gdal.GDT_Float32)
    mean.GetRasterBand(1).WriteArray(ndvi_mean)
    ndvi_std = np.std(ndvi_month,0)
    std = driver.Create(idir+'NDVI_STD\\STD_' + week +'.tif', ndvi_std.shape[1], ndvi_std.shape[0], 1, gdal.GDT_Float32)
    std.GetRasterBand(1).WriteArray(ndvi_std)
    print(week)
    mean.FlushCache()
    std.FlushCache()
    mean.SetGeoTransform(geoTransform)
    mean.SetProjection(projection)
    std.SetGeoTransform(geoTransform)
    std.SetProjection(projection)
    del ndvi_mean, ndvi_std

01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [2]:
gws_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_final\\' + '*.tif')
gws_list.sort()
mean_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_MEAN\\' + '*.tif')
mean_list.sort()
std_list = glob.glob('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_STD\\' + '*.tif')
std_list.sort()

In [4]:
driver = gdal.GetDriverByName("GTiff")
for i in range(52):
    if i+1 < 10:
        week = '0'+str(i+1)
    else:
        week = str(i+1)
    gws_mean = gdal.Open(mean_list[i])
    gws_mean_arr = gws_mean.ReadAsArray()
    gws_std = gdal.Open(std_list[i])
    gws_std_arr = gws_std.ReadAsArray()
    for j in range(21):
        gws = gdal.Open(gws_list[52*j+i])
        gws_arr = gws.ReadAsArray()
        gws_arr[gws_arr<-2] = np.nan
        projection = gws.GetProjectionRef()
        geoTransform = gws.GetGeoTransform()
        gws_zscore = (gws_arr-gws_mean_arr)/gws_std_arr
        gws_zscore[gws_zscore<-10] == np.nan
        gws_zscore[gws_zscore>10] == np.nan
        year = str(2000+j)
        zscore = driver.Create('R:\\UnitedStates\\ForDRI_project-NEW\\New_Model_2022\\NDVI\\NDVI_zscore\\NDVI_' + year+week+'.tif', gws_arr.shape[1], gws_arr.shape[0], 1, gdal.GDT_Float32)
        zscore.GetRasterBand(1).WriteArray(gws_zscore)
        zscore.GetRasterBand(1).SetNoDataValue(np.nan)
        zscore.SetGeoTransform(geoTransform)
        zscore.SetProjection(projection)
        zscore.FlushCache()
        print('NDVI_' + year+week)

NDVI_200001
NDVI_200101
NDVI_200201
NDVI_200301
NDVI_200401
NDVI_200501
NDVI_200601
NDVI_200701
NDVI_200801
NDVI_200901
NDVI_201001
NDVI_201101
NDVI_201201
NDVI_201301
NDVI_201401
NDVI_201501
NDVI_201601
NDVI_201701
NDVI_201801
NDVI_201901
NDVI_202001
NDVI_200002
NDVI_200102
NDVI_200202
NDVI_200302
NDVI_200402
NDVI_200502
NDVI_200602
NDVI_200702
NDVI_200802
NDVI_200902
NDVI_201002
NDVI_201102
NDVI_201202
NDVI_201302
NDVI_201402
NDVI_201502
NDVI_201602
NDVI_201702
NDVI_201802
NDVI_201902
NDVI_202002
NDVI_200003
NDVI_200103
NDVI_200203
NDVI_200303
NDVI_200403
NDVI_200503
NDVI_200603
NDVI_200703
NDVI_200803
NDVI_200903
NDVI_201003
NDVI_201103
NDVI_201203
NDVI_201303
NDVI_201403
NDVI_201503
NDVI_201603
NDVI_201703
NDVI_201803
NDVI_201903
NDVI_202003
NDVI_200004
NDVI_200104
NDVI_200204
NDVI_200304
NDVI_200404
NDVI_200504
NDVI_200604
NDVI_200704
NDVI_200804
NDVI_200904
NDVI_201004
NDVI_201104
NDVI_201204
NDVI_201304
NDVI_201404
NDVI_201504
NDVI_201604
NDVI_201704
NDVI_201804
NDVI_201904
NDVI

In [6]:
ndvi_arr.shape

(1092, 208, 464)